In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

### Basic Runnable Sequence: Input → Runnable1 → Runnable2 → ... → Output

In [ ]:
from langchain_core.runnables import RunnableLambda

chain = RunnableLambda(lambda x: x + 1) | RunnableLambda(lambda x: x * 2)

chain.invoke(5)
# Output: 12

### Simple LLM Chain: Input → Prompt Template → LLM → Output

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

prompt = ChatPromptTemplate.from_messages([
    ("system", "Just Return Plain Paragraph withput any newline characters in it."),
    ("human","Tell me a joke about {topic}")
])

llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    temperature=0.8, 
    top_p=0.95,
    max_new_tokens=512,
)
model = ChatHuggingFace(llm=llm)

chain = prompt | model

chain.invoke({"topic": "cats"}).content

### With Output Parser: Input → Prompt Template → LLM → Output Parser → Output

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "Just Return Plain Paragraph withput any newline characters in it."),
    ("human","Tell me a joke about {topic}")
])

llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    temperature=0.8, 
    top_p=0.95,
    max_new_tokens=512,
)
model = ChatHuggingFace(llm=llm)
parser = StrOutputParser()

chain = prompt | model | parser

chain.invoke({"topic": "dogs"})

### RAG Retrieval Chain: Input → Retriever → (Context + Question) → Prompt Template → LLM → Output

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace, HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

text_1 = "LangChain is great for building LLM apps."

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_texts(texts=[text_1], embedding=embeddings)
retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_template("Answer based on context:\n{context}\nQuestion: {question}")

llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    temperature=0.8,
    top_p=0.95,
    max_new_tokens=512,
)
model = ChatHuggingFace(llm=llm)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("What is LangChain and who invented it?")

### Parallel (RunnableParallel): Input → [Branch1 → Output1, Branch2 → Output2, ...] → Merged Dict → Next → Output

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    temperature=0.8,
    top_p=0.95,
    max_new_tokens=512,
)
model = ChatHuggingFace(llm=llm)

joke = ChatPromptTemplate.from_template("Tell a joke about {topic}") | model
poem = ChatPromptTemplate.from_template("Write a 2-line poem about {topic}") | model

parallel = RunnableParallel(joke=joke, poem=poem)

parallel.invoke({"topic": "bears"})

### With Passthrough: Input → RunnableParallel(Passthrough + Transform) → Prompt Template → LLM → Output

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

prompt = ChatPromptTemplate.from_template("Original: {original}\nModified: {modified}")

llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    temperature=0.8,
    top_p=0.95,
    max_new_tokens=512,
)
model = ChatHuggingFace(llm=llm)

chain = (
    RunnableParallel(original=RunnablePassthrough(), modified=lambda x: x.upper())
    | prompt
    | model
)

chain.invoke("hello world")

### Custom Function (RunnableLambda): Input → Custom Function → Next → Output

In [ ]:
from langchain_core.runnables import RunnableLambda

def add_ten(x):
    return x + 10

def multiply_by_two(x):
    return x * 2

chain = RunnableLambda(add_ten) | RunnableLambda(multiply_by_two)

chain.invoke(5)
# Output: 30

### Branching and Merging: Input → RunnableParallel(LLM Branch + Other Branch) → Combine Prompt → LLM → Output

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    temperature=0.8,
    top_p=0.95,
    max_new_tokens=512,
)
model = ChatHuggingFace(llm=llm)

branch1 = RunnableLambda(lambda x: f"Joke: {x}")
branch2 = RunnableLambda(lambda x: f"Poem: {x}")

parallel = RunnableParallel(joke=branch1, poem=branch2)

combine_prompt = ChatPromptTemplate.from_template("{joke}\n{poem}\nSummarize both:")
chain = parallel | combine_prompt | model

chain.invoke("cats")

### Itemgetter Routing: Input Dict → Itemgetter(key) → Runnable → Output

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda

chain = (
    {"topic": itemgetter("input")}
    | RunnableLambda(lambda d: d["topic"].upper())
)

chain.invoke({"input": "langchain"})
# Output: 'LANGCHAIN'

### Conditional (via routing): Input → Router Runnable → Selected Chain → Output

In [1]:
from langchain_core.runnables import RunnableBranch, RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    temperature=0.8,
    top_p=0.95,
    max_new_tokens=512,
)

model = ChatHuggingFace(llm=llm)

general_prompt = ChatPromptTemplate.from_messages([
    ("human", "Answer the following question generally: {question}")
])

math_prompt = ChatPromptTemplate.from_messages([
    ("human", "You are a math expert. Solve the following step-by-step: {question}")
])

general = general_prompt | model
math = math_prompt | model

# Extract the question string first, then branch on the string
extract_question = RunnableLambda(lambda input_dict: input_dict["question"])

branch = RunnableBranch(
    (lambda q: "math" in q.lower() or any(op in q.lower() for op in ["+", "-", "*", "/", "calculate", "solve", "="]), math),
    general
)

chain = extract_question | branch

# Test
print(chain.invoke({"question": "What is 2+2?"}))

# print(chain.invoke({"question": "Tell me about quantum physics"}))

g:\GenAI - CampusX\gen_ai_campusx\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


content="Certainly! Let's solve the problem step-by-step.\n\n1. **Identify the Numbers and Operation**: The expression given is 2 + 2. Here, we have two numbers (2 and 2) and an addition operation (+).\n\n2. **Understand the Addition Operation**: Addition is an operation that combines the quantities of two or more numbers. When we add two numbers, we are essentially counting how many items we have in total if we put both groups together.\n\n3. **Perform the Addition**: \n   - Start with the first number: 2\n   - Add the second number: 2 more\n\n4. **Count the Total**: \n   - If you have 2 items and you add 2 more items, you end up with 4 items in total.\n\n5. **Conclusion**: The result of 2 + 2 is 4.\n\nSo, the final answer is **4**." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 188, 'prompt_tokens': 49, 'total_tokens': 237}, 'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b8